# <div align="center">Predict Car Traffic Injury</div>



In this document we follow the CRISP-DM process and methodolgies...


# 1. CRISP-DM: Business Understanding

bla bla bla


<span style="color:cyan">


| Column Name                    | Type           | Short Description                                                                                                                         | Domain Concept           |
|--------------------------------|----------------|-------------------------------------------------------------------------------------------------------------------------------------------|-------------------------|
| CRASH_RECORD_ID                | Text           | Unique identifier for the crash record, linking to the same crash in the Vehicles and People datasets.                                    | Identification          |
| CRASH_DATE_EST_I               | Text           | Indicates if the crash date was estimated by a desk officer or reporting party.                                                           | Crash Details           |
| CRASH_DATE                     | DateTime       | Date and time of the crash as entered by the reporting officer.                                                                           | Crash Details           |
| POSTED_SPEED_LIMIT             | Number         | Posted speed limit as determined by the reporting officer.                                                                                | Environmental Factors   |
| TRAFFIC_CONTROL_DEVICE         | Text           | Traffic control device present at the crash location as determined by the reporting officer.                                              | Environmental Factors   |
| DEVICE_CONDITION               | Text           | Condition of the traffic control device as determined by the reporting officer.                                                           | Environmental Factors   |
| WEATHER_CONDITION              | Text           | Weather condition at the time of the crash as determined by the reporting officer.                                                        | Environmental Factors   |
| LIGHTING_CONDITION             | Text           | Lighting condition at the time of the crash as determined by the reporting officer.                                                       | Environmental Factors   |
| FIRST_CRASH_TYPE               | Text           | Type of first collision in the crash.                                                                                                     | Crash Details           |
| TRAFFICWAY_TYPE                | Text           | Trafficway type as determined by the reporting officer.                                                                                   | Environmental Factors   |
| LANE_CNT                       | Number         | Total number of through lanes in either direction, excluding turn lanes, at the crash location as determined by the reporting officer.     | Environmental Factors   |
| ALIGNMENT                      | Text           | Street alignment at the crash location as determined by the reporting officer.                                                            | Environmental Factors   |
| ROADWAY_SURFACE_COND           | Text           | Road surface condition at the crash location as determined by the reporting officer.                                                      | Environmental Factors   |
| ROAD_DEFECT                    | Text           | Road defects present at the crash location as determined by the reporting officer.                                                        | Environmental Factors   |
| REPORT_TYPE                    | Text           | Administrative report type of the crash.                                                                                                   | Documentation          |
| CRASH_TYPE                     | Text           | General severity classification for the crash.                                                                                             | Crash Details           |
| INTERSECTION_RELATED_I         | Text           | Indicates if an intersection played a role in the crash as observed by the police officer.                                                | Environmental Factors   |
| NOT_RIGHT_OF_WAY_I             | Text           | Indicates if the crash began or first contact was made outside of the public right-of-way.                                                | Legal                   |
| HIT_AND_RUN_I                  | Text           | Indicates if the crash involved a driver who fled the scene without exchanging information and/or rendering aid.                          | Legal                   |
| DAMAGE                         | Text           | Estimated damage from the crash as observed in the field.                                                                                 | Crash Outcome           |
| DATE_POLICE_NOTIFIED           | DateTime       | Calendar date on which the police were notified of the crash.                                                                             | Documentation          |
| PRIM_CONTRIBUTORY_CAUSE        | Text           | Primary factor contributing to the cause of the crash as determined by officer judgment.                                                  | Crash Analysis         |
| SEC_CONTRIBUTORY_CAUSE         | Text           | Secondary factor contributing to the cause of the crash as determined by officer judgment.                                                | Crash Analysis         |
| STREET_NO                      | Number         | Street address number of the crash location as determined by the reporting officer.                                                       | Location Details       |
| STREET_DIRECTION               | Text           | Street address direction (N, E, S, W) of the crash location as determined by the reporting officer.                                       | Location Details       |
| STREET_NAME                    | Text           | Street name of the crash location as determined by the reporting officer.                                                                 | Location Details       |
| BEAT_OF_OCCURRENCE             | Number         | Chicago Police Department beat ID where the crash occurred.                                                                               | Administrative         |
| PHOTOS_TAKEN_I                 | Text           | Indicates if photos were taken at the crash location by the Chicago Police Department.                                                    | Documentation          |
| STATEMENTS_TAKEN_I             | Text           | Indicates if statements were taken from units involved in the crash.                                                                      | Documentation          |
| DOORING_I                      | Text           | Indicates if the crash involved a vehicle occupant opening a door into the path of a bicyclist.                                           | Crash Type             |
| WORK_ZONE_I                    | Text           | Indicates if the crash occurred in an active work zone.                                                                                   | Environmental Factors   |
| WORK_ZONE_TYPE                 | Text           | Type of work zone, if any, where the crash occurred.                                                                                      | Environmental Factors   |
| WORKERS_PRESENT_I              | Text           | Indicates if construction workers were present in the work zone at the crash location.                                                    | Environmental Factors   |
| NUM_UNITS                      | Number         | Number of units involved in the crash, representing different modes of traffic with independent trajectories.                             | Crash Details           |
| MOST_SEVERE_INJURY             | Text           | Most severe injury sustained by any person involved in the crash.                                                                         | Injury Analysis        |
| INJURIES_TOTAL                 | Number         | Total number of people sustaining fatal, incapacitating, non-incapacitating, and possible injuries as determined by the reporting officer. | Injury Analysis        |
| INJURIES_FATAL                 | Number         | Total number of people sustaining fatal injuries in the crash.                                                                            | Injury Analysis        |
| INJURIES_INCAPACITATING        | Number         | Total number of people sustaining incapacitating/serious injuries in the crash.                                                           | Injury Analysis        |
| INJURIES_NON_INCAPACITATING    | Number         | Total number of people sustaining non-incapacitating injuries in the crash.                                                               | Injury Analysis        |
| INJURIES_REPORTED_NOT_EVIDENT  | Number         | Total number of people sustaining possible injuries in the crash as determined by the reporting officer.                                  | Injury Analysis        |
| INJURIES_NO_INDICATION         | Number         | Total number of people with no injuries in the crash as determined by the reporting officer.                                              | Injury Analysis        |
| INJURIES_UNKNOWN               | Number         | Total number of people for whom the injury status, if any, is unknown.                                                                    | Injury Analysis        |
| CRASH_HOUR                     | Number         | Hour of the day when the crash occurred, derived from CRASH_DATE.                                                                         | Time Details           |
| CRASH_DAY_OF_WEEK              | Number         | Day of the week when the crash occurred, derived from CRASH_DATE. Sunday=1                                                                | Time Details           |
| CRASH_MONTH                    | Number         | Month when the crash occurred, derived from CRASH_DATE.                                                                                   | Time Details           |
| LATITUDE                       | Number         | Latitude of the crash location as determined by the reporting officer.                                                                    | Geographic Information |
| LONGITUDE                      | Number         | Longitude of the crash location as determined by the reporting officer.                                                                   | Geographic Information |
| LOCATION                       | Point          | Geographic location of the crash as determined by the reporting officer, allowing for mapping and geographic analysis.                    | Geographic Information |


<br>
<br>
<br>

</span>


<span style="color:cyan">



### **Domain Concepts Explained**

##### **1. Identification**
- **Description**: Features that uniquely identify the crash records or events, often used for linking data across different datasets.
- **Example**: 
  - **CRASH_RECORD_ID**: Serves as a unique identifier for each crash event, enabling connections with related records in vehicles and people datasets.

##### **2. Crash Details**
- **Description**: Specific details about the crash event, including the type of crash, date, and time, providing basic information on how and when the crash occurred.
- **Example**: 
  - **CRASH_DATE**: Provides the exact date and time the crash happened.
  - **FIRST_CRASH_TYPE**: Describes the initial type of collision.

##### **3. Environmental Factors**
- **Description**: Conditions surrounding the crash, such as weather, lighting, and road conditions, which can influence the occurrence and severity of crashes.
- **Example**: 
  - **WEATHER_CONDITION** and **LIGHTING_CONDITION**: Detail the environmental state at the time of the crash, affecting driving conditions and visibility.

##### **4. Legal**
- **Description**: Features related to legal aspects of the crash, including compliance with right-of-way rules and hit-and-run incidents.
- **Example**: 
  - **HIT_AND_RUN_I**: Indicates whether a driver involved in the crash fled the scene without providing information or rendering aid.

##### **5. Crash Outcome**
- **Description**: Descriptions of the immediate consequences of the crash, primarily in terms of physical damage.
- **Example**: 
  - **DAMAGE**: Estimates the monetary damage to property resulting from the crash.

##### **6. Documentation**
- **Description**: Administrative details about the crash reporting and documentation process, including whether photos or statements were taken.
- **Example**: 
  - **DATE_POLICE_NOTIFIED**: Records when the police were officially notified about the crash.

##### **7. Crash Analysis**
- **Description**: Factors identified as contributing to the crash, including primary and secondary causes as determined by officer judgment.
- **Example**: 
  - **PRIM_CONTRIBUTORY_CAUSE** and **SEC_CONTRIBUTORY_CAUSE**: Identify the main reasons behind the crash according to the reporting officer.

##### **8. Location Details**
- **Description**: Specifics about where the crash occurred, including street names and numbers, helping in pinpointing the exact crash location.
- **Example**: 
  - **STREET_NAME**, **STREET_NO**, and **STREET_DIRECTION**: Provide a detailed description of the crash site.

##### **9. Administrative**
- **Description**: Information related to police and emergency response jurisdictions, such as police beats.
- **Example**: 
  - **BEAT_OF_OCCURRENCE**: Refers to the Chicago Police Department beat where the crash took place.

##### **10. Crash Type**
- **Description**: Categories that describe specific scenarios or types of crashes, including incidents involving bicycles or work zones.
- **Example**: 
  - **DOORING_I**: Specifies if the crash involved a vehicle door being opened in the path of a bicyclist.

##### **11. Injury Analysis**
- **Description**: Detailed records of injuries resulting from the crash, categorizing the severity and type of injuries sustained.
- **Example**: 
  - **INJURIES_TOTAL**: Counts the total number of injuries.
  - **INJURIES_FATAL**: Counts the number of fatal injuries.

##### **12. Time Details**
- **Description**: Temporal information about the crash, including the hour, day of the week, and month when the crash occurred.
- **Example**: 
  - **CRASH_HOUR**, **CRASH_DAY_OF_WEEK**, and **CRASH_MONTH**: Provide insights into the timing patterns of crashes.

##### **13. Geographic Information**
- **Description**: Geospatial data related to the crash location, enabling mapping and location-based analysis.
- **Example**: 
  - **LATITUDE** and **LONGITUDE**: Give precise coordinates of the crash site.
  - **LOCATION**: Offers a mappable point of the incident.

Each domain concept helps categorize the dataset's features for easier analysis, highlighting different factors and details that contribute to a comprehensive understanding of traffic crashes.


<br>
<br>
<br>

</span>

<br>
<br>
<br>

# 2. CRISP-DM: Data Understanding

In this section, the following will be addressed:<br><br>
**2.1. Collect Initial Data**<br>
2.1.1. Import the relevant Python packages that are going to be used.<br>
2.1.2. Acquire the data.<br>
2.1.3. Record of the data acquisition process.<br>
2.2.4. HTML Reprot.<br>
<br>
<br>

**2.2. Data Description**<br>
2.2.1. Shape of the Dataset.<br>
2.2.2. Head snippet.<br>
2.2.3. Dataset info.<br>
<br>
<br>

**2.3. Explore the Data**<br>
2.3.1. Basic Statistics - For numeric models, to calculate Mean, Median, mode, ... <br>
2.3.2. Tabular Report - For Continuos and Categorical Features (Refer to HTML Reports please). <br>
2.3.3. Correlations + Heat Map.<br>
<br>
<br>

**2.4. Verify Data Quality**<br>
2.4.1. Completnesss - Missing Values Summary.<br>
2.4.2. Irregular cardinality, (1, too high for categorical, too low for continuos)<br>
2.4.3. Consistency - Handle outliers, out of range data or invalid formats (if any).<br>
2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data.<br>


<br>
<br>

## 2.1. Collect Initial Data


This a generic intial step, to start working on the data:<br>
2.1.1. Import the relevant Python packages that are going to be used.<br>
2.1.2. Acquire the data.<br>
2.1.3. Record of the data acquisition process.<br>

In [ ]:
# 2.1.1. Import the relevant Python packages that are going to be used

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')


In [ ]:
# 2.1.2. Acquire the data

# A generic utilies file with generic functionallity
from misc.utilities import acquire_dataset

# Download & load the dataset
# If the file already downloaded, it will not be re-downloaded (to speed up work efficiency)
df = acquire_dataset()
# df

<span style="color:cyan">


2.1.3. Record of the data acquisition process.<br>

The data acquired from: https://catalog.data.gov/dataset/traffic-crashes-crashes
<br>
The above website belongs to USA government free datasets advised by the course instructions.

<br>
<br>
<br>

</span>


In [ ]:
# Helper functions

def get_continuous_feature_details(my_df, my_feature):
    my_df[my_feature] = pd.to_numeric(my_df[my_feature], errors='coerce')
    # Printing unique values
    unique_values = my_df[my_feature].unique()
    sorted_unique_values = np.sort(unique_values)
    len_unique_values = len(unique_values)
    unique_values_str = ', '.join(map(str, sorted_unique_values[:5])) + ', ..., ' + ', '.join(map(str, sorted_unique_values[-5:]))
    max_value = my_df[my_feature].max()
    min_value = my_df[my_feature].min()
    # print(f"{my_feature}: Unique Len={len_unique_values}, Unique={sorted_unique_values}, Max={max_value}, Min={min_value}")
    print(f"{my_feature}:\n  Unique Len={len_unique_values}\n  Unique={unique_values_str}\n  Max={max_value}\n  Min={min_value}")

    return len_unique_values, sorted_unique_values, max_value, min_value
    

def dump_feature_frequency_to_a_file(my_df, my_feature):
    value_counts = my_df[my_feature].value_counts()
    # value_counts.to_csv(f"{my_feature}_value_counts.csv", index=True, header=['Frequency'])
    # Open a file to write
    with open(f"{my_feature}_value_counts.txt", 'w') as file:
        # Write a header row
        file.write(f"{'Value':<20}{'Frequency':<10}\n")
        file.write(f"{'-'*20}{'-'*10}\n")
        
        # Iterate over the Series and write each value and its frequency
        for value, count in value_counts.items():
            file.write(f"{value:<20}{count:<10}\n")



<br>
<br>

## 2.2. Data Description

In this phase we investigate the following aspects:<br>
2.2.1. Shape of the Dataset.<br>
2.2.2. Head snippet.<br>
2.2.3. Dataset info.<br>
2.2.4. Numerical Feature Distribution.<br>
2.2.5. Catagorical Feature Distribution.<br>
2.2.6 HTML report<br>

In [ ]:
# 2.2.1. Shape of the Dataset.

def get_dataset_shape(df):
    # Create a new DataFrame to display the shape information
    shape_df = pd.DataFrame({
        'Aspect': ['Records (Instances)', 'Features (Columns)'],
        'Number': [df.shape[0], df.shape[1]]
    })
    return shape_df

# Display the DataFrame
get_dataset_shape(df)


In [ ]:
# 2.2.2. Head snippet

def get_dataset_head(df):
    df.head()

get_dataset_head(df)


In [ ]:
# 2.2.3. Dataset info

def get_dataset_info(df):
    df.info()

get_dataset_info(df)


In [ ]:
# 2.2.4. Numerical Feature Distribution

# Filter out deprecated warnings from Seaborn
warnings.filterwarnings("ignore")

# Select only numeric columns for distribution plots
numeric_df = df.select_dtypes(include=[np.number])

# Determine the number of rows/columns for the subplot grid
num_features = numeric_df.shape[1]
num_rows = int(np.ceil(num_features / 3))  # Adjust the denominator to change the number of columns

# Create a figure and a grid of subplots
plt.figure(figsize=(15, num_rows * 5))

for i, column in enumerate(numeric_df.columns):
    plt.subplot(num_rows, 3, i + 1)
    sns.histplot(numeric_df[column], kde=True, stat = 'density')

    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')

# Adjust layout for better visualization
plt.tight_layout()
plt.show()

In [ ]:
## 2.2.5 Catagorical Feature Distribution

# Select only categorical columns
categorical_df = df.select_dtypes(include=['object', 'category']) #exclude=['number']

# Delete some of the columns due ot cardinality issues ( they have alot of levels like Id)
categorical_df = categorical_df.drop(['PRIM_CONTRIBUTORY_CAUSE','SEC_CONTRIBUTORY_CAUSE','STREET_NAME','DATE_POLICE_NOTIFIED','LOCATION','CRASH_RECORD_ID' , 'CRASH_DATE_EST_I','CRASH_DATE'] , axis = 1)

# Determine the number of rows/columns for the subplot grid
num_features = categorical_df.shape[1]
num_rows = int(np.ceil(num_features / 3))  # Adjust for desired number of columns per row

plt.figure(figsize=(15, num_rows * 5))

for i, column in enumerate(categorical_df.columns):
    plt.subplot(num_rows, 3, i + 1)
    ax = sns.countplot(y=categorical_df[column])
    total = len(categorical_df[column])  # Total number of data points for the percentage calculation
    for p in ax.patches:
        percentage = '{:.1f}%'.format(100 * p.get_width()/total)
        x = p.get_x() + p.get_width() + 0.02  # Shifts the text to the right side of the bars
        y = p.get_y() + p.get_height()/2
        ax.annotate(percentage, (x, y))
    
    plt.title(f'Distribution of {column}')
    plt.xlabel('Count')
    plt.ylabel(column)

plt.tight_layout()
plt.show()


<br>
<br>

## 2.3. Explore the Data

2.3.1. Basic Statistics - For numeric models, to calculate Mean, Median, mode, ... <br>
2.3.2. Tabular Report - For Continuos and Categorical Features (Refer to HTML Reports please). <br>
2.3.3. Correlations + Heat Map.<br>


In [ ]:
# 2.3.1. Continuous Features Tabular Report

def get_continuous_features_tabular_report(df):
    # continuous_features_tabular_report_df = df.describe(include=['number'])
    return df.describe(include=['number'])

# Transpose - More friendly print
continuous_features_transpose_tabular_report_df = get_continuous_features_tabular_report(df).transpose()

continuous_features_transpose_tabular_report_df


In [ ]:
# 2.3.2. Categorical Features Tabular Report

def get_categorical_features_tabular_report(df):
    return df.describe(exclude=['number'])

# Transpose - More friendly print
categorical_features_transpose_tabular_report_df = get_categorical_features_tabular_report(df).transpose()

categorical_features_transpose_tabular_report_df



In [ ]:
# 2.3.3. Correlation insights + HeatMap

def get_correlation_insights(df):
    # # Get the relevant columns
    subset_df = df[get_continuous_features_tabular_report(df).columns]

    # Calculate the correlation matrix
    # correlation_matrix = subset_df.corr()
    return subset_df.corr()

correlation_matrix = get_correlation_insights(df)
correlation_matrix

# # Create a heatmap using seaborn
# plt.figure(figsize=(16, 14))
# sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.8)

In [ ]:
# 2.3.4. HeatMap

def draw_heatmap(correlation_matrix):
    # Create a heatmap using seaborn
    plt.figure(figsize=(16, 14))
    return sns.heatmap(correlation_matrix, cmap='coolwarm', annot=True, fmt=".2f", linewidths=0.8)

draw_heatmap(correlation_matrix)

<br>
<br>

## 2.4. Verify Data Quality

2.4.1. Completnesss - Missing Values Summary.<br>
2.4.2. Irregular cardinality, could be one of the following cases:<br>
* Features with a cardinality of 1.
* Too high cardinality for categorical features.
* Too low cardinality for continous features.<br>

2.4.3. Consistency - Handle outliers.<br>
2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data.<br>



In [ ]:
# 2.4.1. Completnesss - Missing Values Summary

# Get missing values details
# Retrun value per feature as a numeric value and a percentage
def get_missing_values_details(df):
    # Check for missing values in each column
    missing_values = df.isnull().sum()
    missing_values_sorted = missing_values.sort_values(ascending=False)
    # Check the percentage of missing values for each column
    missing_percentage_sorted = (missing_values_sorted / len(df)) * 100
    return missing_values_sorted, missing_percentage_sorted


def get_completeness_report(df):
    missing_values_sorted, missing_percentage_sorted = get_missing_values_details(df)
    # Create a DataFrame to summarize the completeness
    completeness_report = pd.DataFrame({
        'Missing Values': missing_values_sorted,
        'Missing Percentage': missing_percentage_sorted
    })
    return completeness_report

completeness_report = get_completeness_report(df)
completeness_report


In [ ]:
# Section x.y. - Check Ranges & Special Outliers
get_continuous_feature_details(df, 'CRASH_HOUR')
get_continuous_feature_details(df, 'CRASH_DAY_OF_WEEK')
get_continuous_feature_details(df, 'CRASH_MONTH')
get_continuous_feature_details(df, 'LATITUDE')
get_continuous_feature_details(df, 'LONGITUDE')
pass

<span style="color:cyan">

##### x.y. Check Ranges & Special Outliers
##### Analysis

This section addresses problems mostly in ranges for special features.
For example, week can't be more than 7 days.
Let's go over the descriptive features and provide insights.



| Feature            | Unique Values Len | Unique Values Example    | Max Value | Min Value | Comments                |
|--------------------|-------------------|--------------------------|-----------|-----------|-------------------------|
| CRASH_HOUR         | 24 |  [0 1 2 ... 23]  |  23 | 0 | Time of the crash (0-23)|
| CRASH_DAY_OF_WEEK  | 7 | [1 2 3 4 5 6 7] | 7 | 1 | Day of the week (1-7)   |
| CRASH_MONTH        | 12 | [1 2 3 ... 12] |  12  |  1   | Month of the crash (1-12)|
| LATITUDE           |  299299 |  [ 0 41.64467013 41.64469152 ... 42.02273632 ]      |    42.022779861       |     0.0      | Geographical latitude   |
| LONGITUDE          |  299262  | [-87.93619295 -87.93587692 ... 0] |   0.0        |     -87.936192947      | Geographical longitude  |

<br>

As the table illustrates, the following features have valid values:<br>
* CRASH_HOUR
* CRASH_DAY_OF_WEEK
* CRASH_MONTH

<br>

But the following features has values out of range / outliers in respect to each one type:
* LATITUDE - Min value is 0.0
* LONGITUDE - Max value is 0.0

<br>

Doing research in Google Maps and Google for Chicago city LATITUDE and LONGITUDE range (also NaN, but will be addressing in missing values section), here is our findings:<br>
LATITUDE valid range: [41.640, 42.023]<br>
LONGITUDE valid range: [-87.940, -87.524]<br>

We will address that again in the Data Preprocessing section to clean up the out of range values.

<br>
<br>
<br>

</span>

In [ ]:
# 2.4.2. Irregular cardinality - Features with a cardinality of 1

IRREGULAR_CARD_SINGLE = 1             # To detect features with cardinality of 1

def get_irregular_cardinality_with_single_cardinality(df):
    unique_counts = df.nunique()
    # for feature_name, count in unique_counts.items():
    #     if count == 1:
    #         print(f"The feature '{feature_name}' has a unique count of 1 - Irregular Cardinality Issue")
    unique_counts_single_threshold = unique_counts[unique_counts == IRREGULAR_CARD_SINGLE]
    return pd.DataFrame(unique_counts_single_threshold)


get_irregular_cardinality_with_single_cardinality(df)


In [ ]:
# 2.4.2. Irregular cardinality - Too high cardinality for categorical features

IRREGULAR_CARD_MAX_THRESHOLD = 10     # To detect categorical features with high cardinality

def get_unique_counts_exceed_max_threshold_categorical(df):
    cardinality = df.select_dtypes(exclude=['number']).apply(lambda x: x.nunique())
    high_cardinality_cols = cardinality[cardinality > IRREGULAR_CARD_MAX_THRESHOLD].index.tolist()
    # print("High Cardinality Columns:", high_cardinality_cols)
    return high_cardinality_cols

high_cardinality_cols = get_unique_counts_exceed_max_threshold_categorical(df)
# Categorical Features with carinality higher than IRREGULAR_CARD_MAX_THRESHOLD
print(f"Categorical Features with carinality higher than {IRREGULAR_CARD_MAX_THRESHOLD} cardinality are: {high_cardinality_cols}")

In [ ]:
# 2.4.2. Irregular cardinality - Too low cardinality for continuous features

IRREGULAR_CARD_MIN_THRESHOLD = 10      # To detect continuous features with low cardinality

def get_unique_counts_exceed_min_threshold_continuous(df):
    continuous = df.select_dtypes(include=['number']).apply(lambda x: x.nunique())
    high_cardinality_cols = continuous[continuous < IRREGULAR_CARD_MIN_THRESHOLD].index.tolist()
    # print("High Cardinality Columns:", high_cardinality_cols)
    return high_cardinality_cols
    # cardinality = df.select_dtypes(exclude=['number']).apply(lambda x: x.nunique())
    # unique_counts = df.nunique()
    # unique_counts_below_min_threshold = unique_counts[unique_counts < IRREGULAR_CARD_MIN_THRESHOLD]
    # return pd.DataFrame(unique_counts_below_min_threshold)

get_unique_counts_exceed_min_threshold_continuous(df)


<span style="color:cyan">

##### 2.4.2.  - Irregular Cardinality
##### Analysis

To do - describe the results from above.

Explain why ABOVE NUMBERS ARE OK.

<br>
<br>
<br>

</span>

In [ ]:
# 2.4.3. Consistency - Handle outliers

def get_outliers_list_of_indices(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    ls = df.index[ (df[ft] < lower_bound) | (df[ft] > upper_bound) ]
    
    return ls

# To store the indicies
outliers_index_list = []
# To store the features (set, so no duplications)
outliers_features_set = set()
numeric_columns = df.select_dtypes(include=['number']).columns

for feature in numeric_columns:
    indicies_list = get_outliers_list_of_indices(df, feature)
    outliers_index_list.extend(indicies_list)
    if len(indicies_list) != 0:
        outliers_features_set.add(feature)

print(f"There are {len(outliers_index_list)} outliers")
print(f"Features set are {outliers_features_set}")

<span style="color:cyan">

##### 2.4.2.  - Outliers
##### Analysis

Outliers inital result shows the following features are outliers:
* STREET_NO
* POSTED_SPEED_LIMIT
* INJURIES_REPORTED_NOT_EVIDENT
* LONGITUDE
* LANE_CNT
* BEAT_OF_OCCURRENCE
* INJURIES_TOTAL
* INJURIES_INCAPACITATING
* INJURIES_NON_INCAPACITATING
* INJURIES_NO_INDICATION
* LATITUDE
* INJURIES_FATAL
* NUM_UNITS

Let's understand if they are really outliers, or the values are valid (e.g. street number is valid to be in a very big range).

<br>
<br>
<br>

</span>

In [ ]:
get_continuous_feature_details(df, 'STREET_NO')
get_continuous_feature_details(df, 'POSTED_SPEED_LIMIT')
get_continuous_feature_details(df, 'INJURIES_REPORTED_NOT_EVIDENT')
get_continuous_feature_details(df, 'LONGITUDE')
get_continuous_feature_details(df, 'LANE_CNT')
get_continuous_feature_details(df, 'BEAT_OF_OCCURRENCE')
get_continuous_feature_details(df, 'INJURIES_TOTAL')
get_continuous_feature_details(df, 'INJURIES_INCAPACITATING')
get_continuous_feature_details(df, 'INJURIES_NON_INCAPACITATING')
get_continuous_feature_details(df, 'INJURIES_NO_INDICATION')
get_continuous_feature_details(df, 'LATITUDE')
get_continuous_feature_details(df, 'INJURIES_FATAL')
get_continuous_feature_details(df, 'NUM_UNITS')
pass # To avoid printing of the previous line and keep only formatted-pretty prints

<span style="color:cyan">

##### 2.4.2.  - Outliers
##### Analysis

| Feature                        | Unique Values Len | Unique Values Example                  | Max Value    | Min Value    | Comments                                                                 | Explanation                                        |
|--------------------------------|-------------------|----------------------------------------|--------------|--------------|--------------------------------------------------------------------------|----------------------------------------------------|
| STREET_NO                      | 11723             | 0, 1, 2, 3, 4, ..., 13787, 13795, 13799, 34453, 451100 | 451100       | 0            | Street numbers in Chicago, large range due to city size and data errors. | Street values can highly vary, it doesn't follow math logic      |
| POSTED_SPEED_LIMIT             | 46                | 0, 1, 2, 3, 4, ..., 62, 63, 65, 70, 99 | 99           | 0            | Posted speed limits, including possibly erroneous values.                | Speed limits can highly vary. We checked all values count to verify no outliers in respect to allowed speed - e.g. no instance with value 500 (please refer to the function dump_feature_frequency_to_a_file)       |
| INJURIES_REPORTED_NOT_EVIDENT  | 14                | 0.0, 1.0, 2.0, 3.0, 4.0, ..., 9.0, 10.0, 11.0, 15.0 | 15.0         | 0.0          | Number of non-evident injuries reported per crash.                       | Range is reasonable for severe accidents.           |
| LONGITUDE                      | 299263            | -87.936192947, ..., -87.524587387, 0.0 | 0.0          | -87.936192947| Longitude values in Chicago, including erroneous 0.0 entries.            | 0.0 likely represents missing or incorrectly entered data. It will be handled in data preprocessing phase. |
| LANE_CNT                       | 42                | 0.0, 1.0, 2.0, 3.0, 4.0, ..., 1191625.0| 1191625.0    | 0.0          | Number of lanes, with some unrealistic values due to errors.             | Extreme max value due to data entry errors.        |
| BEAT_OF_OCCURRENCE             | 277               | 111.0, ..., 2535.0, 6100.0             | 6100.0       | 111.0        | Police beat codes, including possibly erroneous 6100.0.                 | This is police internal codes and can't be treated as outliers          |
| INJURIES_TOTAL                 | 21                | 0.0, 1.0, ..., 19.0, 21.0              | 21.0         | 0.0          | Total number of injuries per crash.                                      | Range is within expected limits for traffic accidents. |
| INJURIES_INCAPACITATING        | 11                | 0.0, 1.0, ..., 8.0, 10.0               | 10.0         | 0.0          | Number of incapacitating injuries per crash.                             | Within expected limits, considering severe cases.   |
| INJURIES_NON_INCAPACITATING    | 20                | 0.0, 1.0, ..., 19.0, 21.0              | 21.0         | 0.0          | Number of non-incapacitating injuries per crash.                         | Range reflects possible severe accidents.           |
| INJURIES_NO_INDICATION         | 49                | 0.0, 1.0, ..., 50.0, 61.0              | 61.0         | 0.0          | People involved in crash with no injuries reported.                      | High values for large accidents, not outliers.     |
| LATITUDE                       | 299300            | 0.0, 41.644670132, ..., 42.022779861   | 42.022779861 | 0.0          | Latitude values in Chicago, including erroneous 0.0 entries.             | 0.0 likely represents missing or incorrectly entered data. It will be handled in data preprocessing phase. |
| INJURIES_FATAL                 | 6                 | 0.0, 1.0, ..., 3.0, 4.0                | 4.0          | 0.0          | Number of fatal injuries per crash.                                      | Fatalities are rare, but values are within expected limits. |
| NUM_UNITS                      | 17                | 1, 2, ..., 16, 18                      | 18           | 1            | Number of units involved in a crash (e.g. cars, motocycles, ...) | Not an outlier, the highest values sounds reasonable |

From the above table, we can notice that the following features has outliers:
* LANE_CNT
* LONGITUDE
* LATITUDE


<br>
<br>
<br>

</span>


In [ ]:
# 2.4.4. Uniqueness - Remove duplicated or irrelevant repetition in data

def consistency_check_duplicated_instances(df):
    duplicate_rows = df.duplicated().sum()
    print(f"Number of duplicate rows: {duplicate_rows}")
    # If you want to actually see the duplicate rows, you can use:
    if duplicate_rows > 0:
        print(df[df.duplicated(keep=False)].sort_values(by=df.columns.tolist()))

consistency_check_duplicated_instances(df)


<br>
<br>
<br>

# 3. CRISP-DM: Data Preparation

3.1. Missing Values - Drop or Imputation.<br>
3.2. Irregular Cardinality.<br>
3.3. Outliers - To remove instances.<br>
3.4. Consistency - Invalid formats / Ranges.<br>

More details in each section.

In [ ]:
# 3.1. Missing Values (Part-1) - Drop

# For each step, we take a copy. It is easier to debug and investigate.
df_drop_missing_features = df.copy()

# Set the threshold for dropping columns
MISSING_THRESHOLD = 30.0

# Identify columns that have missing value percentage greater than the threshold
columns_to_drop = completeness_report[completeness_report['Missing Percentage'] >= MISSING_THRESHOLD].index

# Drop these columns from the DataFrame
df_drop_missing_features = df_drop_missing_features.drop(columns=columns_to_drop)

# Now, df has the columns dropped where the missing value percentage was higher than 30%
print("Features with missing instances higher than 30% that has been dropped")
columns_to_drop

In [ ]:
# 3.1. Missing Values (Part-2) - Special handle for LATITUDE and LONGITUDE

df_drop_missing_instances = df_drop_missing_features.copy()


def drop_missing_instances(my_df, my_feature):
    # Remove rows where LATITUDE is NaN
    df_cleaned = my_df.dropna(subset=[my_feature])
    return df_cleaned
drop_missing_instances(df_drop_missing_features, 'LATITUDE')
drop_missing_instances(df_drop_missing_features, 'LATITUDE')
pass


In [ ]:
# 3.1. Missing Values (Part-2) - Imputation for values


df_imputation_missing = df_drop_missing_instances.copy()

# Identify columns that have missing value percentage less than the imputation threshold
columns_to_impute = completeness_report[completeness_report['Missing Percentage'] < MISSING_THRESHOLD].index


# Loop through the columns and perform imputation
for column in columns_to_impute:
    # if df_prepared[column].dtype == 'float64' or df[column].dtype == 'int64':
    if df_imputation_missing[column].dtype == 'numeric':
        # Impute numerical columns with the mean value
        df_imputation_missing[column].fillna(df_imputation_missing[column].mean(), inplace=True)
    else:
        # Impute categorical columns with the mode value (the most frequent value)
        df_imputation_missing[column].fillna(df_imputation_missing[column].mode()[0], inplace=True)


In [ ]:
# 3.1. Missing Values (Part-3) - Print for visibility of current situation

completeness_report = get_completeness_report(df_imputation_missing)
completeness_report

<span style="color:cyan">

##### 3.1. Missing Values -Imputation for values
##### Analysis

# To-Do - Update about the three parts

For missing values higher than 30%, we drop them.
For missing values lower than 30% - we do imputation. To be more accurate, in the terms of our dataset, missing values under 30% ranges between 0.1%-3% missing values so imputation is a very reasonable choice in this case.

Please refer to missing values report in the Data Understanding section.

<br>
<br>
<br>

</span>

In [ ]:
# x.y. Drop Duplicated features

df_drop_location = df_imputation_missing.copy()

df_drop_location = df_drop_location.drop('LOCATION', axis=1)



<span style="color:cyan">

##### x.y. Duplicated Columns - LOCATION is the pair LATITUDE and LONGITUDE
##### Analysis

LOCATION is the pair of values of LONGITUDE and LATITUDE, we choose to drop it due to two reasons:
1. It is duplication of other features (aggregation of LONGITUDE and LATITUDE in pairs will result in LOCATION).
2. It is categorical feature, with high class dimensionality - meaning, in the encoding phase to prepare for modeling, it will generate very big number of derived features. In the other hand, LONGITUDE and LATITUDE is numeric and much easier to the handling in modeling phase.

<br>
<br>
<br>

</span>

In [ ]:
# x.y. Drop Duplicated features

df_drop_crash_record = df_drop_location.copy()
df_drop_crash_record = df_drop_crash_record.drop('CRASH_RECORD_ID', axis=1)


<span style="color:cyan">

##### x.y. Drop CRASH_ID column
##### Analysis

It is just ID of the crash, and has no contributation to the prediction model.

<br>
<br>
<br>

</span>

In [ ]:
# Section x.y. - Fix Ranges and Special Outliers

df_fix_range_issues = df_drop_crash_record.copy()

print("========================== Values Before ==========================")
get_continuous_feature_details(df_fix_range_issues, 'LATITUDE')
get_continuous_feature_details(df_fix_range_issues, 'LONGITUDE')
# get_continuous_feature_details(df_fix_range_issues, 'LANE_CNT')

# To remove values for my_feature outside a given range range (including min_value and max_value)
def remove_invalid_values_outside_given_range(my_df, my_feature, min_value, max_value):
    # Create a mask for values within the range
    valid_mask = (my_df[my_feature] >= min_value) & (my_df[my_feature] <= max_value)
    # Apply the mask to the DataFrame
    filtered_df = my_df[valid_mask]
    return filtered_df

# Define the valid range for LATITUDE
min_latitude = 41.640
max_latitude = 42.023
df_fix_range_issues = remove_invalid_values_outside_given_range(df_fix_range_issues, 'LATITUDE', min_latitude, max_latitude)

# Define the valid range for LONGITUDE
min_longitude = -87.940
max_longitude = -87.524
df_fix_range_issues = remove_invalid_values_outside_given_range(df_fix_range_issues, 'LONGITUDE', min_longitude, max_longitude)


# Already dropped due to high missing values count
# min_lane_count = 0
# max_lane_count = 10
# df_fix_range_issues = remove_invalid_values_outside_given_range(df_fix_range_issues, 'LANE_CNT', min_lane_count, max_lane_count)


print("========================== Values After ==========================")
get_continuous_feature_details(df_fix_range_issues, 'LATITUDE')
get_continuous_feature_details(df_fix_range_issues, 'LONGITUDE')
# get_continuous_feature_details(df_fix_range_issues, 'LANE_CNT')

get_continuous_features_tabular_report(df_fix_range_issues).transpose()


<span style="color:cyan">

##### x.y. Fix Ranges and Special Outliers
##### Analysis

LOCATION is the pair of values of LONGITUDE and LATITUDE, we choose to drop it due to two reasons:
1. It is duplication of other features (aggregation of LONGITUDE and LATITUDE in pairs will result in LOCATION).
2. It is categorical feature, with high class dimensionality - meaning, in the encoding phase to prepare for modeling, it will generate very big number of derived features. In the other hand, LONGITUDE and LATITUDE is numeric and much easier to handling in modeling phase.

<br>
<br>
<br>

</span>

In [ ]:
# x.y. - Fix Ranges, special outliers and formats
# Please refer to Data Understanding section or to my next comment, I have summarized all here for visibility


In [ ]:

df_values_count = df_fix_range_issues.copy()

dump_feature_frequency_to_a_file(df_values_count ,'POSTED_SPEED_LIMIT')
dump_feature_frequency_to_a_file(df_values_count ,'DAMAGE')
dump_feature_frequency_to_a_file(df_values_count ,'CRASH_TYPE')
dump_feature_frequency_to_a_file(df_values_count ,'REPORT_TYPE')


In [ ]:
# 3.3. Outliers - To remove instances.

df_clean_outliers = df_fix_range_issues.copy()

def outliers1(df, ft):
    Q1 = df[ft].quantile(0.25)
    Q3 = df[ft].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    ls = df.index[ (df[ft] < lower_bound) | (df[ft] > upper_bound) ]
    return ls

index_list = []
for feature in df_clean_outliers.columns:
    if pd.api.types.is_numeric_dtype(df_clean_outliers[feature]):
        index_list.extend(outliers1(df_clean_outliers, feature))

def remove(df, ls):
    ls = sorted(set(ls))
    df = df.drop(ls)
    return df

print(f"There are {len(set(index_list))} outliers")
print(f"DataFrame Shape Before Removing Outliers {df_clean_outliers.shape}")
df_clean_outliers = remove(df_clean_outliers, index_list)
print(f"DataFrame Shape After  Removing Outliers {df_clean_outliers.shape}")


<span style="color:cyan">

##### 3.3. Outliers - To remove instances
##### Analysis

To-Do

Please refer to the outliers section report in the Data Understanding section.

</span>

In [ ]:
# 3.4. Consistency - Invalid formats / Ranges

df_cleaned = df_clean_outliers.copy()

# Convert CRASH_DATE to datetime
df_cleaned['CRASH_DATE'] = pd.to_datetime(df_cleaned['CRASH_DATE'])

# Extract components from CRASH_DATE
df_cleaned['YEAR'] = df_cleaned['CRASH_DATE'].dt.year

In [ ]:
print(df_cleaned.shape)
df.shape

# 4. CRISP-DM: Modeling

**Section Overview**

4.1. Import relevant ML libs for Modeling.<br>
4.2. Formaluize a Numerical Measure for the Target Variable(s).<br>
4.3. Undersampling - To fix over-representation in the dataset (unbalanced data).<br>
4.4. Modeling with Random Forest.<br>
4.5. Modeling with Logestic Regression.<br>
4.6. Modeling with DNN.<br>
4.7. Modeling with KNN.<br>

In [ ]:
# 4.1. Import relevant ML libs for Modeling

from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

In [ ]:
# 4.2. Formaluize a Numerical Measure for the Target Variable(s)

df_to_model = df_cleaned.copy()

# Building our numerical metrics for measuring the Severity of injuries based on the reported injuries in the dataset
df_to_model['SEVERITY_OF_INJURIES'] = ((0.3 * df_to_model['INJURIES_NON_INCAPACITATING'] + 
                                 0.6 * df_to_model['INJURIES_INCAPACITATING'] + 
                                 # Assuming we might use 'INJURIES_INCAPACITATING' or another column for fatal injuries representation
                                 0.1 * df_to_model['INJURIES_NO_INDICATION'] + df_to_model['INJURIES_FATAL']) / 
                                ((df_to_model['INJURIES_NON_INCAPACITATING'] + 
                                      df_to_model['INJURIES_INCAPACITATING'] + 
                                      # Again, assuming a placeholder for fatal injuries if needed
                                      df_to_model['INJURIES_NO_INDICATION']+df_to_model['INJURIES_FATAL'])))

# Code to create the "INJURY_CLASS" feature based on "SEVERITY_OF_INJURIES"
df_to_model['INJURY_ClASS'] = df_to_model['SEVERITY_OF_INJURIES'].apply(lambda x: 'HIGH INJURY' if x > 0.2 else 'LIGHT INJURY')

# Code to create the "SEVERITY_CLASS" feature based on "INJURIES_FATAL" and "INJURIES_INCAPACITATING"
df_to_model['SEVERITY_CLASS'] = df_to_model.apply(lambda x: 'HIGH SEVERITY' if x['INJURIES_FATAL'] > 0 or x['INJURIES_INCAPACITATING'] > 0 else 'LOW SEVERITY', axis=1)

# Display the first few rows to see the new feature
df_to_model[['INJURIES_NON_INCAPACITATING','INJURIES_FATAL', 'INJURIES_INCAPACITATING', 'INJURIES_NO_INDICATION', 'SEVERITY_OF_INJURIES','INJURY_ClASS','SEVERITY_CLASS']].head()

<span style="color:cyan">

##### 4.2. Formaluize a Numerical Measure for the Target Variable(s)
##### Analysis

Our target variable is represented via 4 variables were each one giving the following indications:
* INJURIES_NO_INDICATION - No injury or light reported.
* INJURIES_NON_INCAPACITATING - Medium injury.
* INJURIES_INCAPACITATING - Heavy.
* INJURIES_FATAL - Death.

To build a mesurement to estimate the fatality of the crash injury crash, we define a weighted equation for each class and normalize it.


</span>

In [ ]:

# Preparing features and target variables 
X = df_to_model.drop(['SEVERITY_OF_INJURIES','INJURIES_TOTAL' ,'INJURIES_INCAPACITATING', 'MOST_SEVERE_INJURY','INJURY_ClASS','SEVERITY_CLASS',
                  'INJURIES_NON_INCAPACITATING', 'SEVERITY_CLASS','INJURIES_NO_INDICATION','CRASH_DATE',
                  'INJURIES_UNKNOWN', 'DATE_POLICE_NOTIFIED','INJURIES_REPORTED_NOT_EVIDENT','INJURIES_FATAL','CRASH_RECORD_ID','LOCATION','STREET_NAME'],axis = 1)
y = df_to_model['SEVERITY_CLASS']
X = pd.get_dummies(X)

# Initialize the random under-sampler due to unbalanced dataset
rus = RandomUnderSampler(random_state=42)

# Resample the dataset
X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
# 4.4. Modeling with Random Forest

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier with balanced class weights
rf_classifier = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)

# Fitting the model to the training data
rf_classifier.fit(X_train, y_train)

# Predicting on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_results = classification_report(y_test, y_pred)

print("Accuracy of Random Forest Classifier:", accuracy)
print("\nClassification Report:\n", classification_report_results)


### Important Features
based on RF classifier I check the important features for detecting the severity of injuries in the accident.

In [ ]:
importances = rf_classifier.feature_importances_
feature_names = X.columns
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

feature_importances.head()

In [ ]:
# 4.5. Modeling with Logestic Regression

# Initialize the Logistic Regression model with balanced class weights
log_reg = LogisticRegression( max_iter=10000, random_state=42)

# Fitting the model to the training data
log_reg.fit(X_train, y_train)

# Predicting on the test set
y_pred = log_reg.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_results = classification_report(y_test, y_pred)

print("Accuracy of linear regressor:", accuracy)
print("\nClassification Report:\n", classification_report_results)


### Important features

In [ ]:
# Get the feature names
feature_names = X_resampled.columns

# Get the coefficients from the logistic regression model
coefficients = log_reg.coef_[0]  # Assuming binary classification, hence [0]

# Create a series to map feature names to their coefficients
feature_importance = pd.Series(coefficients, index=feature_names)

# Sort the features by their absolute values to see the most significant ones
feature_importance_sorted = feature_importance.abs().sort_values(ascending=False)

feature_importance_sorted.head()

In [ ]:
# 4.6. Modeling with DNN

# It's important to scale your input features for neural networks
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the DNN model
mlp = MLPClassifier(hidden_layer_sizes=(20,20,10), activation='relu', solver='adam',
                    max_iter=100, random_state=42, verbose=False)

# Train the model
mlp.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = mlp.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_results = classification_report(y_test, y_pred)

print("Accuracy of DNN model:", accuracy)
print("\nClassification Report:\n", classification_report_results)

In [ ]:
# 4.7. Modeling with KNN

# Initialize the KNN classifier
# n_neighbors is set to 10 as an example, but you should tune this parameter
knn = KNeighborsClassifier(n_neighbors=10)

# Train the model
knn.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = knn.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_results = classification_report(y_test, y_pred)

print("Accuracy of KNN model:", accuracy)
print("\nClassification Report:\n", classification_report_results)

# 5. CRISP-DM: Evaluation

in this section I just want to check our model performance on the unseen dataset which is all of them are 'LOW SEVERITY" and check if it has good accuracy or not.

In [ ]:

df_to_evaluate = df_to_model.copy()

# Separate instances with "high injury" and "low injury" classes 
high_injury_instances = df_to_evaluate[df_to_evaluate['SEVERITY_CLASS'] == 'HIGH SEVERITY']
low_injury_instances = df_to_evaluate[df_to_evaluate['SEVERITY_CLASS'] == 'LOW SEVERITY']

# Sample the same number of instances with "low injury" class as "high injury" instances
num_high_injury = len(high_injury_instances)
low_injury_sampled = low_injury_instances.sample(n=num_high_injury, random_state=42)

# Check the model's predictions on the remaining instances with "low injury" class
low_injury_remaining = low_injury_instances.drop(low_injury_sampled.index)

# Concatenate the sampled instances of both classes to create a balanced dataset
balanced_data = pd.concat([high_injury_instances, low_injury_sampled])


# Prepare features and target variable
X_train_balanced = balanced_data.drop(['SEVERITY_OF_INJURIES','INJURIES_TOTAL' ,'INJURIES_INCAPACITATING', 'MOST_SEVERE_INJURY','INJURY_ClASS','SEVERITY_CLASS',
                  'INJURIES_NON_INCAPACITATING', 'SEVERITY_CLASS','INJURIES_NO_INDICATION','CRASH_DATE',
                  'INJURIES_UNKNOWN', 'DATE_POLICE_NOTIFIED','INJURIES_REPORTED_NOT_EVIDENT','INJURIES_FATAL','CRASH_RECORD_ID','LOCATION','STREET_NAME'],axis = 1)
y_train_balanced = balanced_data['SEVERITY_CLASS']

X_test_balanced = low_injury_remaining.drop(['SEVERITY_OF_INJURIES','INJURIES_TOTAL' ,'INJURIES_INCAPACITATING', 'MOST_SEVERE_INJURY','INJURY_ClASS','SEVERITY_CLASS',
                  'INJURIES_NON_INCAPACITATING', 'SEVERITY_CLASS','INJURIES_NO_INDICATION','CRASH_DATE',
                  'INJURIES_UNKNOWN', 'DATE_POLICE_NOTIFIED','INJURIES_REPORTED_NOT_EVIDENT','INJURIES_FATAL','CRASH_RECORD_ID','LOCATION','STREET_NAME'],axis = 1)

y_test_balanced = low_injury_remaining['SEVERITY_CLASS']


X_train_balanced = pd.get_dummies(X_train_balanced)
X_test_balanced = pd.get_dummies(X_test_balanced)





# Combine the training and testing feature data for consistent one-hot encoding
combined_features = pd.concat([X_train_balanced, X_test_balanced], axis=0)

# Apply get_dummies to the combined dataset
combined_features_encoded = pd.get_dummies(combined_features)

# Now split them back into training and testing sets
X_train_encoded = combined_features_encoded.iloc[:len(X_train_balanced), :]
X_test_encoded = combined_features_encoded.iloc[len(X_train_balanced):, :]

# Ensure the data is scaled
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)


# Train the KNN model on the scaled, balanced training data
knn.fit(X_train_scaled, y_train_balanced)

# Make predictions on the scaled, balanced testing set
y_pred = knn.predict(X_test_scaled)

# Evaluate the predictions
accuracy = accuracy_score(y_test_balanced, y_pred)
classification_report_results = classification_report(y_test_balanced, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report_results)



# 6. CRISP-DM: Deployment

To-Do.